In [2]:
import pandas as pd
df_train = pd.read_csv('./source/datasets/df_treino.csv')
df_test = pd.read_csv('./source/datasets/df_teste.csv')

In [3]:
print(df_train['Resultado'].value_counts())
print(df_test['Resultado'].value_counts())

0    5628
1     377
Name: Resultado, dtype: int64
0    1409
1      90
Name: Resultado, dtype: int64


In [5]:
y_train = df_train['Resultado']
X_train = df_train.drop(['Resultado'], axis=1)


O primeiro passo para o treinamento será instanciar os modelos a serem treinados. Os parâmetros utilizados serão os "default" sugeridos